In [2]:
import pandas as pd
import numpy as np  
import re, json, logging, requests, random
from lib.config import connstr
from lib.context import context
from lib.utils import *  
from lib.issue_provider import issue_provider 
from lib.script_generator import script_generator 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore') 

In [3]:
file=f'{ctx.get_download_path()}\BOD_HVA_2024_PrePopulation_Script.xlsx'  
map=pd.read_excel(file, sheet_name='Section 2 System Cyber Incident', header=1, usecols='A,D',converters={'FY23 #':str,'FY24 #':str} ) 
map = map[map['FY24 #'].notna()]
map 

,FY23 #,FY24 #
1,15,15
10,15.1,15.1
11,NaN,15.2
12,15.2,15.3
13,15.3,15.3.1
14,15.4,15.3.2


In [ ]:
df2023 = sql_todf("  SELECT Idtext, CAST(PK_Question  AS NVARCHAR(9)) AS PK_23, CAST(Q_TypeID AS NVARCHAR(9)) AS Q_Type_23, CAST(FK_PicklistType AS NVARCHAR(9)) AS FK_PicklistType_23 FROM vwQuestions WHERE PK_Form='2023-A-HVA' AND Q_TypeID NOT IN (18, 92, 63, 69, 70)  ", config['connstr'])
df2024 = sql_todf("  SELECT Idtext, CAST(PK_Question  AS NVARCHAR(9)) AS PK_24, CAST(Q_TypeID AS NVARCHAR(9)) AS Q_Type_24, CAST(FK_PicklistType AS NVARCHAR(9)) AS FK_PicklistType_24 FROM vwQuestions WHERE PK_Form='2024-A-HVA' AND Q_TypeID NOT IN (18, 92, 63, 69, 70)  ", config['connstr'])
merged = df2023.merge(map, left_on='Idtext', right_on='FY23 #', how='left')
merged = df2024.merge(merged, left_on='Idtext', right_on='FY24 #', how='left')
merged=merged.fillna('-')
merged[90:125]

In [ ]:
df2023

In [10]:
lst = ['Aaron', 'Adam', 'Adrian', 'Aidan', 'Alan', 'Albert', 'Alex', 'Alexander', 'Alfie', 'Alvin', 'Andrew', 'Anthony', 'Archie', 'Arthur', 'Austin', 'Bailey', 'Barry', 'Ben', 'Benjamin', 'Bennett', 'Blake', 'Bobby', 'Bradley', 'Brandon', 'Brayden', 'Brian', 'Bruce', 'Caleb', 'Callum', 'Cameron', 'Carl', 'Carlos', 'Carter', 'Charlie', 'Chase', 'Chris', 'Christian', 'Christopher', 'Clark', 'Cody', 'Colin', 'Connor', 'Craig', 'Curtis', 'Damian', 'Daniel', 'David', 'Dean', 'Dennis', 'Derek', 'Dominic', 'Don', 'Donald', 'Douglas', 'Dylan', 'Edward', 'Elijah', 'Elliott', 'Eric', 'Ethan', 'Evan', 'Felix', 'Finn', 'Fletcher', 'Francis', 'Frank', 'Fred', 'Gabriel', 'Gavin', 'George', 'Glen', 'Gordon', 'Graham', 'Grant', 'Greg', 'Gregory', 'Harry', 'Harvey', 'Henry', 'Hugh', 'Hunter', 'Ian', 'Isaac', 'Jack', 'Jackson', 'Jacob', 'Jake', 'James', 'Jason', 'Jasper', 'Jayden', 'Jeff', 'Jeffrey', 'Jeremy', 'Jerome', 'Jerry', 'Jesse', 'Jim', 'Jimmy', 'Joe', 'Joel', 'John', 'Johnny', 'Jonathan', 'Jordan', 'Joseph', 'Josh', 'Joshua', 'Julian', 'Justin', 'Keith', 'Ken', 'Kenneth', 'Kevin', 'Kyle', 'Lance', 'Larry', 'Lawrence', 'Lee', 'Leo', 'Leon', 'Liam', 'Logan', 'Louis', 'Lucas', 'Luke', 'Malcolm', 'Marcus', 'Mark', 'Martin', 'Mason', 'Matthew', 'Max', 'Michael', 'Mike', 'Miles', 'Nathan', 'Nathaniel', 'Neil', 'Nicholas', 'Nick', 'Noah', 'Norman', 'Oliver', 'Oscar', 'Owen', 'Patrick', 'Paul', 'Peter', 'Philip', 'Quinn', 'Ralph', 'Raymond', 'Reece', 'Richard', 'Rick', 'Riley', 'Robert', 'Roger', 'Ron', 'Ronald', 'Ross', 'Roy', 'Russell', 'Ryan', 'Sam', 'Samuel', 'Scott', 'Sean', 'Sebastian', 'Seth', 'Shane', 'Shaun', 'Simon', 'Spencer', 'Stanley', 'Stephen', 'Steve', 'Steven', 'Stuart', 'Ted', 'Terry', 'Theo', 'Theodore', 'Thomas', 'Tim', 'Timothy', 'Toby', 'Tony', 'Tyler', 'Victor', 'Vincent', 'Walter', 'Warren', 'Wayne', 'Will', 'William', 'Zachary', 'Zane']
lst
random.shuffle(lst)
', '.join(lst)

'Jacob, David, Clark, Norman, Jerry, Tim, Joe, Oliver, Jimmy, Jesse, Brayden, Leo, Graham, Tony, Steven, Steve, Patrick, Bailey, Sam, Jeffrey, Felix, Henry, Walter, Dean, Elijah, Larry, Jeff, Sebastian, Roger, Seth, John, Brandon, Finn, Logan, Will, Julian, Christian, Vincent, Mike, Arthur, Keith, James, Raymond, Joseph, Jim, Caleb, Andrew, Bennett, Charlie, Alan, Chase, Daniel, Jackson, Kenneth, Shaun, Martin, Mark, Zachary, Reece, Gavin, Dominic, Curtis, Peter, Terry, Callum, Alfie, Richard, Fletcher, Joel, Rick, Ronald, Paul, Damian, Ralph, Carlos, Lance, Connor, Craig, Owen, Jasper, Timothy, Cameron, Ron, Adrian, Neil, Liam, Ethan, Simon, Edward, Marcus, Alex, Nicholas, Dylan, Jason, Hugh, Tyler, Douglas, Wayne, Thomas, Josh, Michael, Lee, Adam, Warren, Carter, Hunter, George, Stuart, Malcolm, Alexander, Anthony, Toby, Donald, Isaac, Colin, Ted, Max, Johnny, Brian, Elliott, Stanley, Dennis, Glen, Ian, Stephen, Russell, Samuel, Joshua, Benjamin, Alvin, Justin, Jordan, Aidan, Victor,

In [ ]:
df = sql_todf(" SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME ='HVAEXT2'  ", config['connstr'])
l=[]
for i,r in df.iterrows():
    l.append(r['COLUMN_NAME'])
print(','.join(l))

In [ ]:
lod=[]
for i in range(0,len(df2024)-1): 
    input=normalize(df2024.iloc[i]['QTEXT'])
    Idtext=df2024.iloc[i]['Idtext']
    for ii,r in df2023[:].iterrows():
        test=normalize(r['QTEXT'] ) 
        if 'PLEASE DESCRIBE' in test:
            continue
        sequence_match = SequenceMatcher(None, test,input).ratio()
        if sequence_match > .75 and Idtext != r['Idtext'] :
            print(r['Idtext'] + ' -> ' + Idtext + ' :' + test)
            lod.append({
                'Idtext_from':r['Idtext'],
                'Idtext_to':Idtext,
                'text':test,
                'sm':sequence_match 
            })

df=pd.DataFrame(lod)

In [ ]:

s='' 
for i,r in df.iterrows(): 
    m23="'"+str(r["FY23 #"])+"'"
    m24="'"+str(r["FY24 #"])+"'"
    s = s + f'( {m23}, {m24} ), \n' 
print( s ) 

In [ ]:
t = zip( list( df['FY23 #'] ), list( df['FY24 #'] ))
output = "".join(t)
output
 

In [ ]:
s1="""		
    UPDATE I SET I.AnsPK_{c} = ANS.AnsPK_{c} FROM ##CIO_CDM_IMPORT I 
    INNER JOIN #CIO_CDM_Data_TMP ANS ON ANS.PK_OrgSubmission = I.PK_OrgSubmission AND ANS.QPK_{c} = I.QPK_{c}"""
s2="""
		UPDATE A Set Answer = CAST(CAST({c} AS DECIMAL(8,2)) AS NVARCHAR(999)) FROM fsma_Answers A INNER JOIN ##CIO_CDM_IMPORT I ON I.AnsPK_{c}=A.PK_Answer ; """
 
s3="""
		SELECT QPK_{c}, PK_OrgSubmission, CAST(CAST({c} AS DECIMAL(8,2)) AS NVARCHAR(999)), GETDATE() FROM ##CIO_CDM_IMPORT WHERE AnsPK_{c} IS NULL
		UNION """
s4="""
			INSERT INTO AuditLog (TableName, PK_PrimeKey, FieldName, FieldValue, Change_Date, UserID, EditType, String_PrimeKey)  
			SELECT 'fsma_Answers', FK_OrgSubmission, QPK_{c}, Answer, @GETDATE, 0, 'D', CAST(PK_Answer as NVARCHAR(99))
			FROM fsma_Answers A
			INNER JOIN ##CIO_CDM_IMPORT I ON I.PK_OrgSubmission=A.FK_OrgSubmission
			AND FK_Question=QPK_{c} 
            """
s5="""          
                --{c}
				SELECT @FK_Question= QPK_{c}, @Answer={c}, @PK_OrgSubmission=PK_OrgSubmission FROM ##CIO_CDM_IMPORT WHERE ROWNUM=@i; 
				SELECT TOP 1 @PK_Answer=PK_Answer FROM #temp_Answers WHERE FK_Question=@FK_Question AND FK_OrgSubmission=@PK_OrgSubmission 
				IF @PK_Answer IS NOT NULL
					BEGIN
						UPDATE fsma_Answers SET Answer=@Answer, DateModifed=@GETDATE WHERE PK_Answer=@PK_Answer
					END
				ELSE
					BEGIN
						INSERT INTO fsma_Answers (FK_Question, FK_OrgSubmission, Answer, DateModifed)
						SELECT QPK_{c}, PK_OrgSubmission, GFE_endpoints, @GETDATE
						FROM ##CIO_CDM_IMPORT WHERE ROWNUM=@i;
					END
"""
lst=['GFE_endpoints','GFE_networking','GFE_inout','GFE_unknown','cisa_7days','cisa_14days','cisa_30days']
lst = [s5.replace('{c}',i)  for i in lst]
print(''.join(lst))

In [ ]:
s = ', '
c =  [ f", @{re.sub('[^A-Za-z0-9]','', c)} NVARCHAR(99)" for c in df.columns ]
print( '\n'.join(c) )

In [ ]:
def formatqt(s):
    s=s.replace('•','') 
    s=s.replace('','<br>')
    s=re.sub('[\n|\r]','',s) 
    return s  
l=formatqt(""" """) 
insert='\nINSERT INTO fsma_Questions (PK_Question, FormName, FK_QuestionGroup,sortpos,FK_PickListType, FK_QuestionType,identifier_text,ExternalLinkType, QuestionText, help_text) VALUES 	\n'

In [ ]:
file=f'{ctx.get_down_dir()}ig.xlsx' 
converts={'Question':lambda x: re.sub('\n','',x) } 
dff=pd.read_excel(file, converters=converts, sheet_name=int(0), header=0, usecols='A,P,Q,R,X,W,Y,U,V,AB,AC,AD,AE,AF,AI,AJ,AK,AL,AO,AQ,AP,AR' ) 
dff[106:110]

In [ ]:
df=dff 
df = df[df["Question"].str.contains("Question") == False]
df["{QuestionText}"]=df["Question"]
 
df=df.fillna('').astype(str) 
 
df['ht']=df['Unnamed: 15']+df['Unnamed: 17']+df['Maturity Level'] 
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['ht'].apply(lambda s: f'<li> <strong>Ad Hoc:</strong>{s}</li>') 
df.drop(columns=['Unnamed: 15','Unnamed: 17','Maturity Level'], inplace=True) 

df['ht']=df['Unnamed: 20']+df['Unnamed: 21']+ df['Unnamed: 22'] + df['Unnamed: 23']+df['Unnamed: 24'] 
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Defined:</strong>{s}</li>')
df.drop(columns=['Unnamed: 20', 'Unnamed: 21','Unnamed: 22', 'Unnamed: 23','Unnamed: 24'], inplace=True)

df['ht']=df['Unnamed: 27']+df['Unnamed: 28']+df['Unnamed: 29'] + df['Unnamed: 30']+df['Unnamed: 31']
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Consistently Implemented:</strong>{s}</li>')
df.drop(columns=['Unnamed: 27','Unnamed: 28','Unnamed: 29','Unnamed: 30','Unnamed: 31'], inplace=True)

df['ht']=df['Unnamed: 34']+ df['Unnamed: 35']+ df['Unnamed: 36'] +df['Unnamed: 37']
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Managed and Measurable:</strong>{s}</li>')
df.drop(columns=['Unnamed: 34', 'Unnamed: 35','Unnamed: 36','Unnamed: 37'], inplace=True)

df['ht']=df['Unnamed: 40']+df['Unnamed: 41']+df['Unnamed: 42']+df['Unnamed: 43'] 
df['ht'].fillna('', inplace=True)
df['{helptext}']=df['{helptext}']+df['ht'].apply(lambda s: f'<li> <strong>Optimized:</strong>{s}</li>')
df.drop(columns=['Unnamed: 40','Unnamed: 41','Unnamed: 42','Unnamed: 43'], inplace=True)
 
df.drop(columns=['ht', 'Question'], inplace=True)

df['{helptext}']=df['{helptext}'].apply(lambda s: f'<br/><br/><h6><b>Maturity Level:</b></h6><ul>{s}</ul>' if len(s) > 210 else '' )

df['{helptext}']=df['{helptext}'].apply(formatqt)
df['{QuestionText}']=df['{QuestionText}'].apply(formatqt)
df['{idt}']=df['{QuestionText}'].apply(lambda s: re.findall('(\d{1,2}.\d?\s{1,4})',s)[0].rstrip(" ").rstrip(".") )
df['{FK_PickListType}']='85' 
df['{FK_QuestionType}']=df['{QuestionText}'].apply(lambda s: '9' if "additional information" in s  else '17')

df['{QuestionText}']=df['{QuestionText}'].apply(lambda s: re.sub('\'','`',s) )
df['{helptext}']=df['{helptext}'].apply(lambda s: re.sub('\'','`',s) )
s=2
sort=0
pk=53009
for i,r, in df.iterrows():
    if re.search('Table \d{1,2}',r['{QuestionText}']):
        s=s+1
        sort=0
        pk=pk+10 
    else:
        if sort > 0:
            df.loc[i, '{pre}']=str('\n,')
        else:
            df.loc[i, '{pre}']=str(insert)
        sort=sort+1
        pk=pk+1
        df.loc[i, '{FK_QuestionGroup}']=f'@PK_QGroup+{s}'
        df.loc[i, '{sortpos}']=str(sort)
        df.loc[i, '{PK_Question}']=str(pk) 
df = df[df["{QuestionText}"].str.contains("Table ") == False]
df['{QuestionText}']=df['{QuestionText}'].apply(lambda s: re.sub('\d{1,2}.\d?\s{1,4}','',s).strip(" ") ) 
df[['{QuestionText}','{helptext}']].to_html('out/script.html', escape=False)


In [ ]:
df

In [ ]:
gen=script_generator(ctx)
code=gen.generate(df, code_template_path=f'{ctx.get_tempalte_dir()}fsma_QuestionsInsert.sql')  

In [ ]:
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{code}' ) 

In [ ]:
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\_5_DBUpdate7.53_IG.sql', 'r', encoding='UTF-8') as f: 
    sql=f.read() 
m=re.search('--PARSEBEGIN(.*)--PARSEEND',sql, flags=re.DOTALL)
sql=sql.replace(m.group(1),code)
with open(ctx.get_dest()+'script.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{sql}' ) 

#with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\_5_DBUpdate7.53_IG.sql', 'w', encoding='UTF-8') as f: 
#    f.write() 
